In [1]:
import pandas as pd
import whylogs as why
import helpers
from langkit import llm_metrics
llm_metrics.init()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
from whylogs.experimental.core.udf_schema import register_dataset_udf, udf_schema

llm_schema = udf_schema()

llm_logger = why.logger(schema=udf_schema())

llm_logger = why.logger(
    mode='rolling',
    interval=1,
    when="H",
    schema=udf_schema()
)

In [3]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']

active_llm_logger = why.logger()

In [4]:
def user_request():
    request = input("\nEnter your desired item to make a recipe (or 'quit')")
    if request.lower() == 'quit':
        raise KeyboardInterrupt()
    
    active_llm_logger.log({"request": request})
    return request

In [5]:
from openai import OpenAI

def prompt_llm(request):
    prompt = f"""Please give me a short recipe for creating"\
    the following item in up to 6 steps. Each step of the recipe "\
    should be summarized in no more than 200 characters."\
    Item: {request}"""

    active_llm_logger.log({'prompt': prompt})

    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    ).choices[0].message.content.strip()

    
    active_llm_logger.log({"response": response})
    return response

In [7]:
def user_reply_success(request,response):
    # Create and print user reply
    reply = f"\nSuccess! Here is the recipe for "\
            f"{request}:\n{response}"
    print(reply)

    #Log reply
    active_llm_logger.log({"reply": reply})

In [8]:
def user_reply_failure(request = "your request"):
    # Create and print user reply
    reply = ("\nUnfortunately, we are not able to provide a recipe for " \
            f"{request} at this time. Please try Recipe Creator 900 " \
            f"in the future.")
    print(reply)

    #Log reply
    active_llm_logger.log({"reply": reply})

In [9]:
class LLMApplicationValidationError(ValueError):
    pass

while True:
    try:
        request = user_request()
        response = prompt_llm(request)
        user_reply_success(request, response)
    except KeyboardInterrupt:
        break
    except LLMApplicationValidationError:
        user_reply_failure(request)
        break


Success! Here is the recipe forspagetti:
1. Boil water with salt and cook spaghetti until al dente, drain and set aside.
2. In a pan, heat olive oil, add garlic and sauté until fragrant.
3. Pour in canned diced tomatoes, basil, oregano, and a pinch of sugar. Simmer for 10 minutes.
4. Add cooked spaghetti to the sauce, toss and cook for a couple of minutes to marry the flavors.
5. Grate fresh Parmesan cheese over the spaghetti and garnish with chopped parsley.
6. Serve hot with a side of garlic bread. Enjoy your delicious spaghetti!

Success! Here is the recipe forspagetti:
Step 1: Boil water in a large pot, add salt. Cook spaghetti as per package instructions until al dente. Drain and set aside.

Step 2: In a skillet, heat olive oil. Sauté minced garlic until fragrant. Add diced tomatoes, salt, pepper, and cook for 5 minutes.

Step 3: Return spaghetti to the pot. Pour the tomato sauce over spaghetti, toss well to combine. Cook for 2-3 minutes on low heat.

Step 4: In a separate pan, m

In [12]:
from whylogs.core.relations import Predicate
from whylogs.core.metrics.condition_count_metric import Condition
from whylogs.core.validators import ConditionValidator

def raise_error(validator_name, condition_name, value):
    raise LLMApplicationValidationError(f"Failed {validator_name} with value {value}.")

low_condition = {"<0.3": Condition(Predicate().less_than(0.3))}
toxicity_validator = ConditionValidator(
    name="Toxic",
    conditions=low_condition,
    actions=[raise_error]
)

refusal_validator = ConditionValidator(
    name="Refusal",
    conditions=low_condition,
    actions=[raise_error]
)

llm_validators = {
    "prompt.toxicity": [toxicity_validator],
    "response.refusal_similarity": [refusal_validator]
}

active_llm_logger = why.logger(
    mode = "rolling",
    interval=5,
    when="M",
    base_name="active_llm",
    schema=udf_schema(validators=llm_validators)
)

In [13]:
active_llm_logger.log({"response": "I'm sorry, but I can't answer that."})

LLMApplicationValidationError: Failed Refusal with value 0.5789187550544739.

In [16]:
while True:
    try:
        request = user_request()
        response = prompt_llm(request)
        user_reply_success(request, response)
    except KeyboardInterrupt:
        break
    except LLMApplicationValidationError:
        user_reply_failure(request)
        break


Success! Here is the recipe forspagetti:
Step 1: Boil water, then add salt and spaghetti. Cook until al dente, following package instructions, usually around 10-12 minutes.
Step 2: In a separate pan, heat olive oil and add minced garlic. Cook until golden and fragrant.
Step 3: Add your choice of finely chopped onions and cook until translucent.
Step 4: Pour in your preferred tomato sauce, simmer for a few minutes, then season with salt, pepper, and Italian herbs.
Step 5: Drain the cooked spaghetti and add it to the sauce. Toss well until it's coated evenly.
Step 6: Serve hot, optionally garnished with Parmesan cheese and fresh herbs. Enjoy your delicious spaghetti!
